# Water Potability Exploratory Notebook

In [1]:
import pyspark
from pyspark.sql import SparkSession, Window, DataFrame
import pyspark.sql.functions as F
import pyspark.sql.types as T
from pyspark.mllib.stat import Statistics
import sys

sys.path.append('/home/jovyan/work')

In [2]:
spark = SparkSession.builder.getOrCreate()

## Load the Dataset

In [3]:
df = spark.read.csv('../data/water_potability.csv',inferSchema=True, header=True)

In [4]:
print('Total record count: {}'.format(df.count()))

Total record count: 3276


### Make a Train/Test Split

In [5]:
df_train, df_test = df.randomSplit([0.7, 0.3], seed=42)

# Save the train and test datasets
df_train.write\
    .format('csv')\
    .mode('overwrite')\
    .option("header", "true")\
    .save('../data/water_potability_train')

df_test.write\
    .format('csv')\
    .mode('overwrite')\
    .option("header", "true")\
    .save('../data/water_potability_test')

# Get rid of df so we don't accidentally use it
del df

## Exploring the Data
### Data Definitions:
 1. pH value: PH is an important parameter in evaluating the acid–base balance of water. It is also the indicator of acidic or alkaline condition of water status. WHO has recommended maximum permissible limit of pH from 6.5 to 8.5. The current investigation ranges were 6.52–6.83 which are in the range of WHO standards.
 
2. Hardness: Hardness is mainly caused by calcium and magnesium salts. These salts are dissolved from geologic deposits through which water travels. The length of time water is in contact with hardness producing material helps determine how much hardness there is in raw water. Hardness was originally defined as the capacity of water to precipitate soap caused by Calcium and Magnesium.

3. Solids (Total dissolved solids - TDS): Water has the ability to dissolve a wide range of inorganic and some organic minerals or salts such as potassium, calcium, sodium, bicarbonates, chlorides, magnesium, sulfates etc. These minerals produced un-wanted taste and diluted color in appearance of water. This is the important parameter for the use of water. The water with high TDS value indicates that water is highly mineralized. Desirable limit for TDS is 500 mg/l and maximum limit is 1000 mg/l which prescribed for drinking purpose.

4. Chloramines: Chlorine and chloramine are the major disinfectants used in public water systems. Chloramines are most commonly formed when ammonia is added to chlorine to treat drinking water. Chlorine levels up to 4 milligrams per liter (mg/L or 4 parts per million (ppm)) are considered safe in drinking water.

5. Sulfate:Sulfates are naturally occurring substances that are found in minerals, soil, and rocks. They are present in ambient air, groundwater, plants, and food. The principal commercial use of sulfate is in the chemical industry. Sulfate concentration in seawater is about 2,700 milligrams per liter (mg/L). It ranges from 3 to 30 mg/L in most freshwater supplies, although much higher concentrations (1000 mg/L) are found in some geographic locations.

6. Conductivity: Pure water is not a good conductor of electric current rather’s a good insulator. Increase in ions concentration enhances the electrical conductivity of water. Generally, the amount of dissolved solids in water determines the electrical conductivity. Electrical conductivity (EC) actually measures the ionic process of a solution that enables it to transmit current. According to WHO standards, EC value should not exceeded 400 μS/cm.

7. Organic_carbon: Total Organic Carbon (TOC) in source waters comes from decaying natural organic matter (NOM) as well as synthetic sources. TOC is a measure of the total amount of carbon in organic compounds in pure water. According to US EPA < 2 mg/L as TOC in treated / drinking water, and < 4 mg/Lit in source water which is use for treatment.

8. Trihalomethanes: THMs are chemicals which may be found in water treated with chlorine. The concentration of THMs in drinking water varies according to the level of organic material in the water, the amount of chlorine required to treat the water, and the temperature of the water that is being treated. THM levels up to 80 ppm is considered safe in drinking water.

9. Turbidity: The turbidity of water depends on the quantity of solid matter present in the suspended state. It is a measure of light emitting properties of water and the test is used to indicate the quality of waste discharge with respect to colloidal matter. The mean turbidity value obtained for Wondo Genet Campus (0.98 NTU) is lower than the WHO recommended value of 5.00 NTU.

10. Potability: Indicates if water is safe for human consumption where 1 means Potable and 0 means Not potable.

### View the Schema

In [6]:
df_train.printSchema()

root
 |-- ph: double (nullable = true)
 |-- Hardness: double (nullable = true)
 |-- Solids: double (nullable = true)
 |-- Chloramines: double (nullable = true)
 |-- Sulfate: double (nullable = true)
 |-- Conductivity: double (nullable = true)
 |-- Organic_carbon: double (nullable = true)
 |-- Trihalomethanes: double (nullable = true)
 |-- Turbidity: double (nullable = true)
 |-- Potability: integer (nullable = true)



In [7]:
print('Train record count: {}'.format(df_train.count()))

Train record count: 2353


### View Sample Rows

In [8]:
df_train.show(5)

+----+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+----------+
|  ph|          Hardness|            Solids|       Chloramines|           Sulfate|      Conductivity|    Organic_carbon|  Trihalomethanes|         Turbidity|Potability|
+----+------------------+------------------+------------------+------------------+------------------+------------------+-----------------+------------------+----------+
|null|  98.3679148956603| 28415.57583214058|10.558949998467961|  296.843207792478|505.24026927891407|12.882614472289333|85.32995534051292| 4.119087300328971|         1|
|null|103.46475866009455| 27420.16742458204| 8.417305032089528|              null|485.97450045781375|11.351132730708514| 67.8699636759021| 4.620793451653219|         0|
|null|108.91662923953173|14476.335695268315| 5.398162017711099|  281.198274407849| 512.2323064106689|15.013793389990155| 86.6714587149138| 3.89557206226812

So, this is a binary classification problem with target values of notckd (not chronic kidney disease) and ckd (chronic kidney disease). **Note:** There is an extraneous tab (\t) on at lease one of the target values. We will have to fix that up.

## Target Column
### Target Distribution

In [9]:
freq_table = df_train.select(F.col("Potability").cast("integer")).groupBy("Potability").count().toPandas()
freq_table

,Potability,count
0,1,905
1,0,1448


This is a binary classification problem with a somewhat imbalanced dataset.

## Continuous Variables
### Summary Statistics

In [10]:
df_features = df_train.select('ph','Hardness','Solids',
                'Chloramines','Sulfate','Conductivity',
                'Organic_carbon','Trihalomethanes','Turbidity')
df_features.describe().toPandas().transpose()

,0,1,2,3,4
summary,count,mean,stddev,min,max
ph,1987,7.065394544064872,1.612338730707101,0.0,13.541240236567981
Hardness,2353,196.85970996091234,32.83914120248886,47.432,323.124
Solids,2353,22131.281786788568,8719.781557904313,320.942611274359,61227.19600771213
Chloramines,2353,7.134664863542833,1.5939471833043641,0.3520000000000003,13.127000000000002
Sulfate,1812,333.8051408041043,41.550506072620976,129.00000000000003,481.03064230599716
Conductivity,2353,425.74124559272485,80.696216405619,181.483753985146,753.3426195583046
Organic_carbon,2353,14.237178459474823,3.272659356030207,2.1999999999999886,27.00670661116601
Trihalomethanes,2229,66.27958491438079,16.241327199504088,8.175876384274268,120.03007700530675
Turbidity,2353,3.969157865709428,0.7823760214428234,1.45,6.739


### Null Counts

In [11]:
for col in df_features.columns:
    print('{} Null Count: {}'.format(col, df_features.where(F.col(col).isNull()).count()))

ph Null Count: 366
Hardness Null Count: 0
Solids Null Count: 0
Chloramines Null Count: 0
Sulfate Null Count: 541
Conductivity Null Count: 0
Organic_carbon Null Count: 0
Trihalomethanes Null Count: 124
Turbidity Null Count: 0


We have nulls to deal with for pH, sulfates, and trihalomethanes.